In [1]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib.ticker as mtick
from matplotlib.cm import get_cmap
# import addcopyfighandler

from IPython.core.display import HTML
from IPython.display import display

import model2 as mod
import model_fsolve as modf

import simu as sim

import fluids as fds
from CoolProp.CoolProp import PropsSI

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import tables

import pprint


In [2]:
# Define the fluid and the conditions
fluid = 'MPG'

p_bar = 2
p = 101325*p_bar  # pressure in Pa
T_C = 25
T = 273.15 + T_C  # temperature in K (25°C)

glycol_rate = 0.4

par = {}

par["eps"] = 0.001

par["rho"] = PropsSI('D', 'P', p, 'T', T, f'INCOMP::{fluid}[{glycol_rate}]') # kg/m3

par["eta"] = PropsSI('V', 'P', p, 'T', T, f'INCOMP::{fluid}[{glycol_rate}]')
par["nu"] = par["eta"]/par["rho"]

In [3]:
# Heat exchanger inputs --------------------------------------------------

# MICOE
# ------

par["N"] = 24

par["Lx"] = 5.886 # m, not used in the system calculation
par["inter_riser"] = 1.2285/par["N"]

par["Ly"] = (par["N"]-1)*[par["inter_riser"]] # m


par["Dx"] = 0.008 # m, not used in the system calculation
par["Din"] = 0.042 # m
par["Dout"] = par["Din"] # m

par["manifold"] = "tubular" # or rectangular

coeff = 1.
par["coeff_Kxin"] = coeff
par["coeff_Kxout"] = coeff
par["coeff_Kyin"] = coeff
par["coeff_Kyout"] = coeff

# Z-type or U-type

# par["ref"] = par["N"]-1
par["ref"] = 0 # U

# Parameters

par["theta"] = 90.

par["rough"] = 0.001 # aluminium = 0.001 mm
# end of inputs

par['Ax'] = math.pi*(par['Dx']/2)**2

# Manifold tubulaire ou rectangulaire

if par['manifold'] == 'tubular':
    par['Ain'] = math.pi*(par['Din']/2)**2
    par['Aout'] = math.pi*(par['Dout']/2)**2
elif par['manifold'] == 'rectangular': # rectangulaire (SPRING ou autre)
    par['Ain'] = par['h_man']*par['l_man']
    par['Aout'] = par['Ain']

par['sch'] = 'exchanger'

# 'Crane' or 'perso'
par['method'] = 'Crane'

In [4]:
# Test d'un seul débit

par['QF'] = 100/3600000 # m3/s (0.000278 m3/s = 1000 L/h) (le premier nombre est en L/h)
# Speed and Reynolds at inlet manifold
par['U'] = par['QF']/par['Ain']
par['Reman'] = par['U']*(par['rho']*par['Din'])/par['eta']
tabl,res = modf.PL_fsolve(par,par['sch'],True)

,Pin,Pout,qx
0,243.421607,2.655669e-27,4.178033
1,243.426093,7.583461e-02,4.175629
2,243.432642,1.483241e-01,4.173679
3,243.441255,2.174715e-01,4.172014
4,243.451932,2.832792e-01,4.170552
5,243.464675,3.457494e-01,4.169251
6,243.479484,4.048841e-01,4.168088
7,243.496363,4.606850e-01,4.167050
8,243.515313,5.131540e-01,4.166130
9,243.536338,5.622926e-01,4.165322


In [5]:
# ist_Q_L = np.array([64,152,245,319,402,449,597,700,800,900,1000,1100,1200,1300,1500]) # L/h
list_Q_L = np.array([272,478,652,856,1094])
list_Q = list_Q_L/3600000 # m3/s
dic = {}
dic["MICOE"] = (sim.modf_parametric_flow_rates(par,list_Q))

7.555555555555556e-05
0.00013277777777777778
0.0001811111111111111
0.00023777777777777778
0.0003038888888888889


In [6]:
list_PL_lin = []
list_PL_bend = []
list_Re = []

for i in range(len(list_Q)):
    u = (list_Q[i]/par["Ax"])/par["N"]

    Re = fds.core.Reynolds(u,par["Dx"],par["rho"],mu=par["eta"]) # viscosité dynamique mu ou eta

    list_Re.append(Re)

    f = fds.friction.friction_factor(Re = Re)
    K = f*(par["Lx"]/par["Dx"])
    list_PL_lin.append(((par["rho"]/2)*K*u**2)/1000) # en kPa

    # K2 = fds.fittings.bend_rounded_Crane(par["Dx"],angle=90.,bend_diameters=2.)

    # list_PL_bend.append((par["rho"]/2)*K2*u**2)

In [7]:
list_Re = np.array(list_Re)

In [8]:
list_Q_M = [272,478,652,856,1094]
list_PL_M = [0.485,0.977,1.336,1.941,2.8]

In [9]:
fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=list_Q_L, y=np.array(dic["MICOE"][0]),
                    mode='markers',
                    name='MICOE model'))
fig.add_trace(go.Scatter(x=list_Q_L, y=list_PL_M,
                    mode='markers',
                    name='MICOE datasheet'))
fig.add_trace(go.Scatter(x=list_Re, y=np.array(list_PL_lin),
                    mode='markers',
                    name='MICOE lin',xaxis='x2'))

    # Set x-axis title
fig.update_xaxes(title_text="Time")

fig.update_layout(
    
    yaxis=dict(
    title='Pressure loss (kPa)'),
    
    xaxis=dict(title="Flow rate (L/h)"),

    xaxis2={'title':'Reynolds','anchor': 'y', 'overlaying': 'x', 'side': 'top'}
    
    )

fig.update_layout(
    xaxis1=dict(range=[list_Q_L.min(), list_Q_L.max()]),
    xaxis2=dict(range=[list_Re.min(), list_Re.max()]),
    )

fig.update_layout(
    autosize=False,
    width=1000,
    height=500,
    margin=dict(
        l=0,
        r=0,
        b=50,
        t=50,
        pad=1
    ),
)

fig.add_annotation(text=f"Temperature = {T-273.15} °C, Pressure = {p_bar} bar, glycol_level = {glycol_rate}%",
                  xref="paper", yref="paper",
                  x=0.05, y=0.9, showarrow=False)

fig.show(renderer="browser")